# CRITICAL: Search + close EXTRA and CLEAN keywords before submitting.

# Project 3: Office Relocation

## Process: 
1. Choose 3 Spanish cities that are popular in the gaming industry to query for locations.
2. Assign values to all requirements, based on employee value.
3. Write formulae to return the shortest distance for all requirements in the 3 cities.
4. Sum all distance values multiplied by requirement weight to determine the lowest proximity score.
5. Bonus: add a parameter to include the value of having nearby gaming compmanies.

## 1. Determine Locations.

Given this map of all gaming companies globally, I have chosen the most gaming-populous cities to locate the spanish branch of my gaming company: Barcelona, Madrid, and Valencia.

https://gamedevmap.com/index.php?country=Spain&state=&city=&query=&type=

## 2. Assign Value to All Requirements.

Weight value of all employeees: Multiply salary (glassdoor avg) by number of employees in said position.

### Employees:
- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President

### Requirements:
- Design companies: Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- School: 30% of the company staff have at least 1 child.
- Startups: Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Starbucks: Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Airport: Account managers need to travel a lot.
- Club: Everyone in the company is between 25 and 40, give them some place to go party.
- Vegan restaurant: The CEO is vegan.
- Basketball Stadium: If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- Dog Groomer: The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

### Scrape Glassdoor for employee salaries:
- add employees to a dataframe.
- url glassdoor {employee name}.
- Web scrape each page for salary, add to df.
- add requirement locations to df.

In [19]:
import pandas as pd

![CLEAN](https://stackoverflow.com/questions/63966750/create-a-pandas-dataframe-from-a-dict-of-uneven-length)

In [33]:
# outline all information to turn into a dataframe
title = ['Designer', 'UI/UX Engineer', 'Frontend Developer', 'Data Engineer', 'Backend Developer', 'Account Manager', 'Maintenance', 'Executive', 'CEO']
salary_k = [30, 33, 33, 44, 39, 25, 20, 60, 80] # Checked on glassdoor. EXTRA: parse through glassdoor to maintain an up-to-date list of employee salaries
amt = [20, 5, 10, 15, 5, 20, 1, 10, 1]
total_salary_score = []
location_req = ['design_companies', 'design_companies', 'startups', 'startups', 'startups', 'airport', 'stadium', 'starbucks', 'vegan_eat']
shared_reqs = ['school', 'club', 'groomer', 0,0,0,0,0,0] # CLEAN: Turn 0 values into null values or skip this row.

# add salary score to dataframe-to-be
for i in range(len(title)): 
    total_salary_score.append(salary_k[i]*amt[i])

# Dataframe structure: employee_title, employee_salary, employees_amt, total_score, location_req
to_df = {'title':title, 'salary (k)':salary_k, 'amt':amt, 'total salary score':total_salary_score, 'location req':location_req, 'shared reqs':shared_reqs}
df = pd.DataFrame.from_dict(to_df)

# visualize dataframe and decide on rankings
df

,title,salary (k),amt,total salary score,location req,shared reqs
0,Designer,30,20,600,design_companies,school
1,UI/UX Engineer,33,5,165,design_companies,club
2,Frontend Developer,33,10,330,startups,groomer
3,Data Engineer,44,15,660,startups,0
4,Backend Developer,39,5,195,startups,0
5,Account Manager,25,20,500,airport,0
6,Maintenance,20,1,20,stadium,0
7,Executive,60,10,600,starbucks,0
8,CEO,80,1,80,vegan_eat,0


## 3. Query the Shortest Distances for All Employee Requirements.

### Process:
- Pass a list of potential office locations.
- Calculate the shortest ditance from each requirement for each potential office.

Dataframe structure: office_location, design_companies, school, startups, starbucks, airport, club, vegan_eat, stadium, groomer

In [ ]:
mad = find_offices('Madrid')
bcn = find_offices('Barcelona')
val = find_offices('Valencia')

mad

In [ ]:
bcn

In [ ]:
val

Communal decisions: dog + club. So how much do they care? 

### Query Requirements: Designers

Query MongoDB for the nearest design companies.
Add map graphic below, keep functions to visualize ina spearate function doc.

### Query Requirements: Schools

Query MongoDB for the nearest schools.

## 4. Determine Proximity Score.

Weight value of all employeees: Multiply salary (glassdoor avg) by number of employees in said position